# Including $ T_2 $ relaxation effects

In this notebook we demonstrate how to include $ T_2 $ relaxation to the tissue models. We recommend first familiarizing yourself with the basic functionality of MICROtool before using this functionality.


## Setting up the dmipy model


In [1]:
from dmipy.core.modeling_framework import MultiCompartmentModel
from dmipy.signal_models.gaussian_models import G1Ball
from pathlib import Path
from microtool.dmipy import DmipyTissueModel
from microtool.tissue_model import RelaxedMultiTissueModel
from microtool.monte_carlo.IO import make_expirement_directories
from microtool.utils.IO import save_pickle

plotdir,modeldir,simdir,schemedir = make_expirement_directories(Path.cwd(),"relaxation_output")

# setting up a simple model
simple_ball = G1Ball(lambda_iso=1.7e-9)

# This step is unnecessary
dmipy_model = MultiCompartmentModel([simple_ball])

microtool_model = DmipyTissueModel(dmipy_model)

model = RelaxedMultiTissueModel([microtool_model], volume_fractions=[1.0], relaxation_times=[10.0])

save_pickle(model, modeldir / "model.pkl")

## Setting up the acquisition scheme

In [2]:
from microtool.dmipy import convert_dmipy_scheme2diffusion_scheme

from microtool.utils.saved_schemes import alexander2008_optimized_directions

# We use an acquisition scheme defined in dmipy
dmipy_scheme = alexander2008_optimized_directions([2, 8, 16])
initial_scheme = convert_dmipy_scheme2diffusion_scheme(dmipy_scheme)

# In order to use this acquisition scheme we need to fix the b0 measurements for dmipy to be able to generate signal
initial_scheme.fix_b0_measurements()
save_pickle(initial_scheme, schemedir / "initial.pkl")
print(initial_scheme)

Acquisition scheme with 44 measurements and 6 scalar parameters:
  DiffusionBValue [s/mm²] in (0.0, 20000.0)    DiffusionGradientAnglePhi [rad] (fixed)    DiffusionGradientAngleTheta [rad] (fixed)    DiffusionPulseWidth [ms] in (1.0, 100.0)    DiffusionPulseInterval [ms] in (1.0, 1000.0)    EchoTime [ms] in (0.1, 1000.0)
-------------------------------------------  -----------------------------------------  -------------------------------------------  ------------------------------------------  ----------------------------------------------  --------------------------------
                                          0                                 -1.96142                                       1.98074                                            7                                              12                           36.2367
                                          0                                  2.96645                                       0.952638                              

Now that we have the acquisition scheme and tissue model defined we test the generation of signal.

In [3]:
microtool_model(initial_scheme)

array([1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.26539597e-01, 1.26539597e-01,
       2.27448870e-03, 2.27448870e-03, 2.27448870e-03, 2.27448870e-03,
       2.27448870e-03, 2.27448870e-03, 2.27448870e-03, 2.27448870e-03,
       1.49871220e-13, 1.49871220e-13, 1.49871220e-13, 1.49871220e-13,
       1.49871220e-13, 1.49871220e-13, 1.49871220e-13, 1.49871220e-13,
       1.49871220e-13, 1.49871220e-13, 1.49871220e-13, 1.49871220e-13,
       1.49871220e-13, 1.49871220e-13, 1.49871220e-13, 1.49871220e-13])

## Optimizing the scheme with respect to the diffusivity.

In [7]:
from microtool.optimize import optimize_scheme

scheme, _ = optimize_scheme(initial_scheme, microtool_model, noise_variance=0.02, method=None)
save_pickle(scheme, schemedir / "optimal.pkl")

C:\Users\Frank\miniconda3\envs\devmicro\lib\site-packages\scipy\optimize\_constraints.py:446: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "
C:\Users\Frank\miniconda3\envs\devmicro\lib\site-packages\dmipy\core\acquisition_scheme.py:179: UserWarning: No b0 measurements were detected. Check if the b0_threshold option is high enough, or if there is a mistake in the acquisition design.
  warn(msg)


## Fitting the model

In [8]:
# Generating basic signal on the new scheme
signal = microtool_model(scheme)

fitted_model = microtool_model.fit(scheme, signal)
fitted_model.fitted_parameters

ValueError: acquisition scheme must have b0-measurements for signal-attenuation-based model fitting.

## Validation of scheme optimization through Monte Carlo simulations

In [ ]:
from microtool.monte_carlo.simulation import MonteCarloSimulation
from scipy import stats

# loc is mean and scale is standard deviation
noise_distribution = stats.norm(loc=0, scale=0.02)

simulation = MonteCarloSimulation(scheme,microtool_model,noise_distribution,n_sim=100)

simulation.run()
simulation.save(simdir / "optimal.pkl")

simulation.set_scheme(initial_scheme)
simulation.run()
simulation.save(simdir / "non_optimal.pkl")

